In [5]:
import wikipedia
import wikipediaapi
import spacy
import re
from collections import Counter
nlp = spacy.load('en_core_web_md')
user_agent = "QuizBowlPractice/1.0 (ani.aniketdixit_00@yahoo.com)"
wiki_wiki = wikipediaapi.Wikipedia(
    language='en', extract_format= wikipediaapi.ExtractFormat.WIKI, user_agent = user_agent)

In [6]:
def get_page(term, clues_list):
    page = wiki_wiki.page(term)
    
    if page.exists():
        if 'may refer to:' in page.text:
            link_list = [l.title for l in page.links.values()]
            
            counts = Counter()
            
            for link in link_list:
                link_page = wiki_wiki.page(link)
                link_text = link_page.text.lower()
                
                for c in clues_list:
                    counts[link] += link_text.count(c.lower())
                    
            best_link = max(link_list, key=lambda link: counts[link])
            return wiki_wiki.page(best_link)
        else:
            return page
        
    

In [49]:
def get_page_clue(clue, main_term):
    page = wiki_wiki.page(clue)
    term_page = wiki_wiki.page(main_term)
    
    if page.exists() and page.title != term_page.title:
        if 'may refer to:' in page.text:
            link_list = [l.title for l in page.links.values()]
            
            counts = Counter()
            
            for link in link_list:
                link_page = wiki_wiki.page(link)
                link_text = link_page.text.lower()
    
                counts[link] += link_text.count(main_term.lower())
                    
            best_link = max(link_list, key=lambda link: counts[link])
            return wiki_wiki.page(best_link)
        else:
            return page
    else:
        search_results = wikipedia.search(clue)

        if len(search_results) > 0:
            if (wiki_wiki.page(search_results[0]).title != term_page.title):
                return wiki_wiki.page(search_results[0])
            else:
                break
        else:
            break

In [50]:
def find_sentences(term, clue_list, clue):

    page_t = get_page(term, clue_list)
    page_c = get_page_clue(clue, term)
    
    matching = []

    page_lower_t = page_t.text
    page_lower_c = page_c.text
    
    Tsentences = re.split(r"(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s", page_lower_t)
    Csentences = re.split(r"(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s", page_lower_c)
    
    for s in Tsentences:
        if clue.lower() in s.lower():
            new_sentence = s.replace(clue, clue.upper())
            matching.append(new_sentence)
        
    for s in Csentences:
        if term.lower() in s.lower():
            new_sentence = s.replace(term, term.upper())
            matching.append(new_sentence)
            
    if matching:
        n = 0
        for m in matching:
            n += 1
            print(f"{n}: {m}" + '\n\n')

In [52]:
find_sentences("Treasure Island", ['Israel Hands', 'Jim Hawkims'], "Israel Hands")

1: He boards the ship and encounters the pirate ISRAEL HANDS, who had been injured in a drunken dispute with one of his companions.


2: ISRAEL HANDS: The ship's coxswain and Flint's old gunner.


3: He is later killed by ISRAEL HANDS in a drunken fight on the Hispaniola.


4: Tom: An honest sailor who is killed by Silver for refusing to join the mutiny.Among other minor characters whose names are not revealed are the four pirates who were killed in an attack on the stockade along with Job Anderson; the pirate killed by the honest men minus Jim Hawkins before the attack on the stockade; the pirate killed by Ben the night before the attack on the stockade; the pirate shot by Squire Trelawney when aiming at ISRAEL HANDS, who later died of his injuries; and the pirate marooned on the island along with Tom Morgan and Dick Johnson.


5: The name "ISRAEL HANDS" was taken from that of a real pirate in Blackbeard's crew, whom Blackbeard maimed (by shooting him in the knee) simply to ensure tha